This notebook will compute the MUR between the BM25 en BERT ranking for the top 100 on the development set.

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
from tqdm.auto import tqdm 
from tqdm import tqdm_notebook

## Load Data

In [4]:
data_dir = 'data/'

# BM25 top 100 ranking
bm25_top100_filename = 'run_development_top100.tsv'

# BERT top 100 ranking
bert_top100_filename = 'bert_thesis_dataset_top100.tsv'

# Experiment query ids
thesis_query_subset = 'experiment_query_subset.tsv'

In [7]:
bm25_df = pd.read_csv(data_dir + bm25_top100_filename,delimiter='\t',encoding='utf-8',header=None)
bm25_df.columns = ['query_id', 'passage_id', 'bm25_rank']

bert_df = pd.read_csv(data_dir + bert_top100_filename,delimiter='\t',encoding='utf-8',header=None)
bert_df.columns = ['query_id', 'passage_id', 'bm25_rank', 'query', 'passage', 'bert_score', 'bert_rank']

query_subset = pd.read_csv(data_dir + thesis_query_subset,delimiter='\t',encoding='utf-8',header=None)
query_subset.columns = ['query_id', 'query']

models_dict = {"bm25": bm25_df, "bert": bert_df}

## Helper Functions

In [9]:
def get_query_ids(dataframe):
    return list(np.unique(dataframe['query_id'].tolist()))

In [ ]:
def get_top_n_ranking(dataframe,rank_column,n):
    top_n_ranking = dataframe[dataframe[rank_column] <= n].sort_values(by=[rank_column])
    return top_n_ranking

In [42]:
def compute_mur(query_id,i):
    matches = 0
    bm25_top_n_passages = bert_df[(bert_df['query_id'] == query_id) & (bert_df['bm25_rank'] <= i)].sort_values(by=['bm25_rank'])['passage_id'].values.tolist()
    bert_top_n_passages = bert_df[(bert_df['query_id'] == query_id) & (bert_df['bert_rank'] <= i)].sort_values(by=['bert_rank'])['passage_id'].values.tolist()
    for index, passage_id in enumerate(bm25_top_n_passages):
        if bert_top_n_passages[index] == passage_id:
            matches += 1
    return matches

In [54]:
avg_murs = {}
for i in range(1,11):
    summed_mur = 0
    for query_id in query_ids:
        mur = compute_mur(query_id,i)
        summed_mur += mur
    avg_mur = summed_mur/len(query_ids)
    avg_murs[i] = round(avg_mur,2)

In [55]:
avg_murs

{1: 0.33,
 2: 0.53,
 3: 0.58,
 4: 0.65,
 5: 0.72,
 6: 0.81,
 7: 0.86,
 8: 0.88,
 9: 0.88,
 10: 0.91}